<a href="https://colab.research.google.com/github/softwarerbfl/TABA_AI_Project/blob/main/AlexNetModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 전처리

In [23]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2
import base64
import zipfile
import imageio
import torch
import h5py
from google.colab import drive
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.autograd import Variable
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [24]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
#저작권 이미지 경로
cpImage_path="/content/gdrive/My Drive/TABA_AI/Image/copyrighted/copyrightedImage_"
#비저작권 이미지 경로
noncpImage_path="/content/gdrive/My Drive/TABA_AI/Image/noncopyrighted/ILSVRC2010_val_"

**train 데이터셋 모으기**

In [7]:
train_cp_path="/content/gdrive/My Drive/TABA_AI/Image/train/cp/"
train_noncp_path="/content/gdrive/My Drive/TABA_AI/Image/train/noncp/"

In [9]:
#저작권 이미지 90개 옮기기
for i in range(90):
  num='{0:08d}'.format(i)
  img_path=cpImage_path+str(num)+".JPEG"
  img=Image.open(img_path)
  img.save(train_cp_path+"copyrightedImage_"+str(num)+'.JPEG')
#비저작권 이미지 90개 옮기기
for i in range(1,91):
  num='{0:08d}'.format(i)
  img_path=noncpImage_path+str(num)+'.JPEG'
  img=Image.open(img_path)
  img.save(train_noncp_path+"noncopyrightedImage_"+str(num)+".JPEG")

**test 데이터셋 모으기**

In [10]:
test_cp_path="/content/gdrive/My Drive/TABA_AI/Image/test/cp/"
test_noncp_path="/content/gdrive/My Drive/TABA_AI/Image/test/noncp/"

In [12]:
#저작권 이미지 10개 옮기기
for i in range(90,100):
  num='{0:08d}'.format(i)
  img_path=cpImage_path+str(num)+".JPEG"
  img=Image.open(img_path)
  img.save(test_cp_path+"copyrightedImage_"+str(num)+'.JPEG')
#비저작권 이미지 90개 옮기기
for i in range(91,101):
  num='{0:08d}'.format(i)
  img_path=noncpImage_path+str(num)+'.JPEG'
  img=Image.open(img_path)
  img.save(test_noncp_path+"noncopyrightedImage_"+str(num)+".JPEG")

**DataLoader로 이미지 데이터 가져오기**

In [25]:
class ImageTransform():

    def __init__(self):
        self.data_transform = transforms.Compose([
            transforms.RandomCrop(256),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    def __call__(self, img):
        return self.data_transform(img)

In [26]:
from torchvision import datasets, transforms, models

train_transforms = transforms.Compose([
                                       #transforms.Resize(224),
                                       transforms.CenterCrop(256),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([
                                      #transforms.Resize(224),
                                      transforms.CenterCrop(256),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# train_data = datasets.ImageFolder(DATASET_PATH + '/train', transform=train_transforms)
# test_data = datasets.ImageFolder(DATASET_PATH + '/test', transform=test_transforms)

# train_iter=torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
# test_iter=torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)

print("Preparing dataset done")

Preparing dataset done


In [27]:
from torchvision.datasets import ImageFolder
train_dataset=ImageFolder("/content/gdrive/My Drive/TABA_AI/Image/train",transform=train_transforms)
test_dataset=ImageFolder("/content/gdrive/My Drive/TABA_AI/Image/test",transform=test_transforms)

In [28]:
train_iter=DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=1)
test_iter=DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=1)

# AlexNet

**모델 선언**

In [29]:
from torch import nn

class AlexNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.CNN=nn.Sequential(
        # MaxPooling층: 데이터의 중요한 요소들만 요약하여 추출
        nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4, padding=0),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(size=5, k=2),
        nn.MaxPool2d(kernel_size=3, stride=2),

        # MaxPooling층: 데이터의 중요한 요소들만 요약하여 추출
        nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),
        nn.ReLU(inplace = True),
        nn.LocalResponseNorm(size=5, k=2),
        nn.MaxPool2d(kernel_size=3, stride=2),

        # 중간다리 역할 
        nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace = True),
        nn.LocalResponseNorm(size=5, k=2),

        # 중간다리 역할 
        nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace = True),
        nn.LocalResponseNorm(size=5, k=2),

        # 중간다리 역할 
        nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace = True),
        nn.LocalResponseNorm(size=5, k=2),

        # MaxPooling층 : Fully Connected 층 2개로 구성 
        nn.MaxPool2d(kernel_size=3, stride=2),
    )
    self.FC = nn.Sequential(
        nn.Linear(256*6*6, 4096),
        nn.ReLU(True),
        nn.Dropout(0.5),

        nn.Linear(4096, 4096),
        nn.ReLU(True),
        nn.Dropout(0.5),

        nn.Linear(4096, 2),
        nn.Softmax()
    )
  def forward(self, inp):
    cnn_res = self.CNN(inp)
    flatten = torch.flatten(cnn_res, 1)
    fc_res = self.FC(flatten)
    return fc_res

In [30]:
model = AlexNet()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

AlexNet(
  (CNN): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU(inplace=True)
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU(inplace=True)
    (6): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (11): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (14): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): LocalResponseNorm(5, alpha=0.0001, beta

**train(훈련) 함수 선언**

In [51]:
def test_eval(model, data_iter, batch_size):
  with torch.no_grad():
    test_loss = 0
    total = 0
    correct = 0
    model.eval()
    for batch_img, batch_lab in data_iter:
      X = batch_img.to(device)
      Y = batch_lab.to(device)
      y_pred = model(X)
      _, predicted = torch.max(y_pred.data, 1)
      correct += (predicted == Y).sum().item()
      total += batch_img.size(0)
    
    # plt.figure(figsize=(5,6))
    # for i in range(20):
    #   plt.subplot(4,5,i+1)
    #   plt.imshow(X[i])
    #   plt.title("real:",Y[i],"predict:",predicted[i])
    #   plt.axis('off')
    # plt.show()
    val_acc = (100*correct / total)
    model.train()
  print("real:",Y)
  print("predict:",predicted)
  return val_acc

In [50]:
from torch import optim
optimizer=optim.Adam(model.parameters(), lr=0.0001)
criterion=nn.CrossEntropyLoss() #
EPOCHS=30
## Training Phase
print_every = 1
print("Start training !")

# Training loop
for epoch in range(30):
  loss_val_sum = 0
  for batch_img, batch_lab in train_iter:

    X = batch_img.to(device)
    Y = batch_lab.to(device)

    # inference & Calculate loss
    y_pred = model.forward(X)
    loss = criterion(y_pred, Y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss_val_sum += loss

  if ((epoch%print_every)==0) or (epoch==(EPOCHS-1)):
    #accr_val = M.test(x_test, y_test, batch_size)
    loss_val_avg = loss_val_sum / len(train_iter)
    accr_val = test_eval(model, train_iter, 32)
    print(f"epoch:[{epoch+1}/{30}] cost:[{loss_val_avg: .3f}] test_accuracy:[{accr_val:.3f}]")

print("Training Done! ")

Start training !
epoch:[1/30] cost:[ 0.577] test_accuracy:[65.979]
epoch:[2/30] cost:[ 0.701] test_accuracy:[73.711]
epoch:[3/30] cost:[ 0.555] test_accuracy:[72.680]
epoch:[4/30] cost:[ 0.549] test_accuracy:[72.680]
epoch:[5/30] cost:[ 0.569] test_accuracy:[75.258]
epoch:[6/30] cost:[ 0.582] test_accuracy:[75.258]
epoch:[7/30] cost:[ 0.562] test_accuracy:[77.320]
epoch:[8/30] cost:[ 0.532] test_accuracy:[74.742]
epoch:[9/30] cost:[ 0.528] test_accuracy:[75.773]
epoch:[10/30] cost:[ 0.596] test_accuracy:[77.320]
epoch:[11/30] cost:[ 0.549] test_accuracy:[71.134]
epoch:[12/30] cost:[ 0.584] test_accuracy:[71.134]
epoch:[13/30] cost:[ 0.616] test_accuracy:[71.649]
epoch:[14/30] cost:[ 0.607] test_accuracy:[68.557]
epoch:[15/30] cost:[ 0.644] test_accuracy:[76.289]
epoch:[16/30] cost:[ 0.589] test_accuracy:[76.289]
epoch:[17/30] cost:[ 0.574] test_accuracy:[74.227]
epoch:[18/30] cost:[ 0.526] test_accuracy:[76.804]
epoch:[19/30] cost:[ 0.511] test_accuracy:[78.351]
epoch:[20/30] cost:[ 0.

**테스트**

In [52]:
model.eval()
accr_val = test_eval(model, test_iter, 32)
print(f"test_accuracy:[{accr_val:.3f}]")

real: tensor([0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0],
       device='cuda:0')
predict: tensor([1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')
test_accuracy:[70.000]
